In [1]:
import requests
import json
import xmltodict
import pandas as pd

import time

In [3]:
df = pd.read_csv(r'data/2023년 1월 서울시 차량통행속도.CSV', encoding='euc-kr')
df = df[['링크아이디', '시점명', '종점명', '방향', '거리', '08시']]
df.columns = ['LINK_ID', '시점명', '종점명', '방향', '거리', '속력']
df['소요시간'] = df['거리'] / (df['속력'] * 1000 / 60)

df_vertex = pd.read_csv(f'data/vertex.csv')
df_vertex.columns = ['idx', 'LINK_ID', 'VERTEX_IDX', 'x', 'y']
df_vertex = df_vertex[['LINK_ID', 'VERTEX_IDX', 'x', 'y']]

df_vertex_start = df_vertex.loc[df_vertex.groupby('LINK_ID').VERTEX_IDX.idxmin()]
df_vertex_start.columns = ['LINK_ID', 'S_V', 'S_X', 'S_Y']
df_vertex_end = df_vertex.loc[df_vertex.groupby('LINK_ID').VERTEX_IDX.idxmax()]
df_vertex_end.columns = ['LINK_ID', 'E_V', 'E_X', 'E_Y']

df_merge = pd.merge(df, df_vertex_start, on='LINK_ID')
df_merge = pd.merge(df_merge, df_vertex_end, on='LINK_ID')

df_coord = pd.read_csv(f'data/coord_start_end.csv')
df_coord = df_coord[['x', 'y', 'lng', 'lat']]
df_coord.columns = ['S_X', 'S_Y', 'S_lng', 'S_lat']
df_merge = pd.merge(df_merge, df_coord, on=['S_X', 'S_Y'])

df_coord.columns = ['E_X', 'E_Y', 'E_lng', 'E_lat']
df_merge = pd.merge(df_merge, df_coord, on=['E_X', 'E_Y'])


In [5]:
df

,LINK_ID,시점명,종점명,방향,거리,속력,소요시간
0,1080012200,아카데미하우스,국립4.19묘지,상행,1179,44.45,1.591451
1,1080012800,국립4.19묘지,국립4.19묘지입구,상행,352,13.50,1.564444
2,1080012700,국립4.19묘지입구,국립4.19묘지,하행,354,21.69,0.979253
3,1080012100,국립4.19묘지,아카데미하우스,하행,1190,38.07,1.875493
4,1230024700,방이초교,방이동고분군,상행,844,26.64,1.900901
...,...,...,...,...,...,...,...
5049,1190003703,중앙대학교,성심의집,하행,398,22.56,1.058511
5050,1190003702,성심의집,이화빌라,하행,143,15.91,0.539283
5051,1190003701,이화빌라,이화약국,하행,133,21.92,0.364051
5052,1130016400,망원정사거리,한강공원입구,상행,692,17.77,2.336522


In [4]:
df_merge

,LINK_ID,시점명,종점명,방향,거리,속력,소요시간,S_V,S_X,S_Y,E_V,E_X,E_Y,S_lng,S_lat,E_lng,E_lat
0,1080012200,아카데미하우스,국립4.19묘지,상행,1179,44.45,1.591451,0,200093,460082,19,200923,460853,127.001054,37.640356,127.010459,37.647302
1,1080012800,국립4.19묘지,국립4.19묘지입구,상행,352,13.50,1.564444,0,200926,460851,2,201263,460952,127.010493,37.647284,127.014312,37.648194
2,1080012700,국립4.19묘지입구,국립4.19묘지,하행,354,21.69,0.979253,0,201260,460964,2,200922,460862,127.014278,37.648302,127.010447,37.647383
3,1080012100,국립4.19묘지,아카데미하우스,하행,1190,38.07,1.875493,0,200922,460862,19,200090,460084,127.010447,37.647383,127.001020,37.640374
4,1230024700,방이초교,방이동고분군,상행,844,26.64,1.900901,0,210686,446132,3,210312,445376,127.120871,37.514605,127.116630,37.507798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5049,1190003703,중앙대학교,성심의집,하행,398,22.56,1.058511,0,196199,445235,11,195894,445066,126.957011,37.506577,126.953562,37.505053
5050,1190003702,성심의집,이화빌라,하행,143,15.91,0.539283,0,195894,445067,1,195752,445052,126.953562,37.505062,126.951956,37.504927
5051,1190003701,이화빌라,이화약국,하행,133,21.92,0.364051,0,195752,445052,1,195620,445038,126.951956,37.504927,126.950464,37.504800
5052,1130016400,망원정사거리,한강공원입구,상행,692,17.77,2.336522,0,191602,450292,3,191075,450741,126.904961,37.552111,126.898992,37.556151


In [ ]:

nodes = []
edges = []

# startable_node dict : check that the node is startable node
startable_node = dict()
ended_node = dict()

for i, row in df_merge.iterrows():
    start_id = i * 2
    end_id = i * 2 + 1

    # marking startable node
    startable_node[start_id] = True
    ended_node[end_id] = True
    
    nodes.append({
        "id": start_id,
        "lat": float(row['S_lat']),
        "lng": float(row['S_lng']),
        "info": {}
    })

    nodes.append({
        "id": end_id,
        "lat": float(row['E_lat']),
        "lng": float(row['E_lng']),
        "info": {}
    })

    edges.append({
        'from': start_id,
        'to': end_id,
        "info": {'id': str(row['LINK_ID']),
                    'weight': int(row['소요시간']) + 1}
    })


# 맨하탄 거리가 0.0002(50m) 이내라면 edge를 생성
dist_criteria = 0.0005
for i in range(len(nodes)):
    for j in range(i + 1, len(nodes)):
        # nearby check
        if abs(nodes[i]['lat'] - nodes[j]['lat']) + abs(nodes[i]['lng'] - nodes[j]['lng']) < dist_criteria:
            
            # Case 1 : i -> j link
            if i in startable_node and j in ended_node:
                edges.append({
                    'from': i,
                    'to': j,
                    "info": {'id': '0000000000',
                                'weight': 1}
                })
            
            # Case 2 : j -> i link
            if j in startable_node and i in ended_node:
                edges.append({
                    'from': j,
                    'to': i,
                    "info": {'id': '0000000000',
                                'weight': 1}
                })

logs = {"nodes": nodes,
        "edges": edges}

# 손으로 전처리 필요
with open('data/seoul_all.json', 'w') as f:
    json.dump(logs, f)